In [2]:
import pandas as pd
import numpy as np

In [3]:
RELATIVE_IN = "2. raw"
RELATIVE_OUT = "3. curated"

## Offers

In [4]:
offers_raw = pd.read_csv(f"{RELATIVE_IN}/offers.csv", index_col=0)
print(offers_raw.shape)
#print(offers_raw.dtypes)

(952, 31)


### Fixing Values

In [5]:
offers_raw["offer uni"].value_counts()

offer uni
The University of Melbourne                        163
Deakin University                                  104
Griffith University                                 84
The University of Western Australia                 61
The University of Notre Dame Sydney                 54
The University of Notre Dame Fremantle              51
Australian National University                      32
The University of Queensland (CQ-WB RMP)            30
The University of Queensland                        29
The University of Wollongong                        27
Macquarie University                                27
The University of Queensland (Greater Brisbane)     14
The University of Queensland (DD MP)                12
Name: count, dtype: int64

In [6]:
UQ_NAME = "The University of Queensland"
RMP_ENDINGS = ["(DD MP)", "(CQ-WB RMP)"]
METRO_ENDING = "(Greater Brisbane)"

# fixing the queensland column
offers = offers_raw.copy()

# get the RMP types
offers.loc[offers["uq type"] == "RMP", "offer uni"] = UQ_NAME + " (RMP)"

# change the MD and WB to correct types
offers.loc[offers["offer uni"].isin([f"{UQ_NAME} {x}" for x in RMP_ENDINGS ]), "offer uni"] = UQ_NAME + " (RMP)"
offers.loc[offers["interview uni"].isin([f"{UQ_NAME} {x}" for x in RMP_ENDINGS ]), "interview uni"] = UQ_NAME + " (RMP)"

# change greater brisbance
metro_mask_offer = (offers["offer uni"] == UQ_NAME) | (offers["offer uni"] == f"{UQ_NAME} {METRO_ENDING}")
metro_mask_interview = (offers["interview uni"] == UQ_NAME) | (offers["interview uni"] == f"{UQ_NAME} {METRO_ENDING}")
offers.loc[metro_mask_offer, "offer uni"] = f"{UQ_NAME} (Metro)"
offers.loc[metro_mask_interview, "interview uni"] = f"{UQ_NAME} (Metro)"

# dropping the type
offers.drop(columns="uq type", inplace=True)

# fixing the interview column
offers.loc[(offers["year"] == 2023) & (offers["interview uni"].isna()), "interviewed?"] = "Yes"
offers.loc[(offers["year"] == 2023) & (offers["interview uni"].notna() & (offers["offer uni"].notna())), "interviewed?"] = "No"

# make the notes lowe case
offers.loc[:, "notes"] = offers["notes"].str.lower()

# drop un necessary columns
offers.drop(columns=["status", "timestamp"], inplace=True)

In [7]:
offers["offer uni"].value_counts()

offer uni
The University of Melbourne               163
Deakin University                         104
Griffith University                        84
The University of Queensland (RMP)         61
The University of Western Australia        61
The University of Notre Dame Sydney        54
The University of Notre Dame Fremantle     51
Australian National University             32
The University of Wollongong               27
Macquarie University                       27
The University of Queensland (Metro)       24
Name: count, dtype: int64

### Getting the marker type column

In [8]:
print(offers[(offers["offer uni place type"].isna()) & (offers["offer uni"].isna())].shape)  # offer uni and offer uni type notna
print(offers[(offers["offer uni"].isna())].shape)                                            # offer uni notna
print(offers[(offers["offer uni place type"].isna())].shape)                                 # offer uni type notna

print(offers[(offers["offer uni"].isna()) & (offers["places selected"].isna())].shape)       # offer uni na with places selected also na

(264, 28)
(264, 28)
(264, 28)
(0, 28)


In [9]:
offers["places selected"].value_counts()

places selected
CSP, BMP          327
CSP, BMP, FFP     119
All                94
CSP & BMP Only     88
CSP                47
CSP Only           29
FFP                 5
BMP                 5
CSP, FFP            5
BMP, FFP            1
Name: count, dtype: int64

In [10]:
# fixing the places selected column with dictionary
CSP = "CSP"
BMP = "BMP"
FFP = "FFP"

rename_places_selected = {
    "CSP, BMP": [CSP, BMP],
    "CSP & BMP Only": [CSP, BMP],

    "CSP, BMP, FFP": [CSP, BMP, FFP],
    "All": [CSP, BMP, FFP],

    "CSP": [CSP],
    "CSP Only": [CSP],
    "CSP, FFP": [CSP, FFP],
    
    "BMP": [BMP],
    "BMP, FFP": [BMP, FFP],

    "FFP": [FFP],
}

offers["places selected"] = offers["places selected"].apply(lambda x: rename_places_selected[x] 
                                                            if (not pd.isna(x)) and (rename_places_selected.get(x)) else x)

offers["places selected"].value_counts()

places selected
[CSP, BMP]         415
[CSP, BMP, FFP]    213
[CSP]               76
[FFP]                5
[BMP]                5
[CSP, FFP]           5
[BMP, FFP]           1
Name: count, dtype: int64

In [11]:
offers['offer uni place type'].value_counts()

offer uni place type
CSP    491
BMP    131
FFP     66
Name: count, dtype: int64

In [12]:
# checks how many times someone gets rejected and the places they selected aren't recorded
offers[offers["places selected"].isna()]["interview uni"].notna().sum()

0

In [13]:
offers["marker"] = offers["offer uni place type"]
offers.loc[offers["marker"].isna(), "marker"] = offers["places selected"]
#offers.loc[offers["marker"].isna(), "marker"] = "Unknown"
offers["marker"].isna().sum()

#offers["marker"].value_counts()

0

### Convert to String

In [14]:
STRING_COLUMNS = ['rurality', 'offer uni preference', 'interview uni preference', 'deakin bonus', 'anu bonus',
                  'mq bonus (gpa)', 'casper quartile', 'uq rmp tier']

for string_col in STRING_COLUMNS:
    offers.loc[:, string_col] = offers[string_col].astype(str)
    offers.loc[:, string_col] = offers.replace('nan', 'None')

C:\Users\mtp63\AppData\Local\Temp\ipykernel_8080\1972849651.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['4.0' '1.0' 'nan' 'nan' 'nan' 'nan' 'nan' '2.0' 'nan' 'nan' 'nan' 'nan'
 '1.0' '1.0' '3.0' '1.0' 'nan' '4.0' '3.0' '1.0' '1.0' '5.0' 'nan' 'nan'
 'nan' 'nan' 'nan' 'nan' '1.0' 'nan' 'nan' 'nan' '5.0' '2.0' 'nan' '1.0'
 '1.0' '1.0' 'nan' 'nan' '1.0' '1.0' 'nan' '1.0' '4.0' 'nan' 'nan' 'nan'
 'nan' '1.0' 'nan' 'nan' 'nan' '1.0' '1.0' 'nan' '1.0' '4.0' '1.0' '1.0'
 'nan' '1.0' '1.0' '2.0' '1.0' '1.0' 'nan' 'nan' 'nan' 'nan' '2.0' '1.0'
 'nan' '1.0' 'nan' '2.0' 'nan' 'nan' '2.0' '2.0' '1.0' 'nan' '1.0' '1.0'
 '2.0' 'nan' 'nan' 'nan' '1.0' '2.0' 'nan' '1.0' 'nan' 'nan' '1.0' '1.0'
 '1.0' 'nan' '1.0' '1.0' 'nan' '1.0' '1.0' '1.0' 'nan' '1.0' '1.0' 'nan'
 '1.0' '2.0' '2.0' '2.0' 'nan' '1.0' '1.0' '1.0' '1.0' '2.0' '1.0' '3.0'
 '1.0' '2.0' 'nan' '1.0' '1.0' '1.0' 'nan' '1.0' 'nan' '1.0' 'nan' 'nan'
 'nan' '2.0' 

### Save CSV

In [15]:
offers.to_csv(f"{RELATIVE_OUT}/offer.csv")

### Messing around

In [16]:
(offers["interview uni"] == UQ_NAME).sum()

0

In [17]:
def uq_mask(series):
    return series.apply(lambda x: UQ_NAME in x if not pd.isna(x) else False)

In [18]:
offers.apply(lambda x: x.count())

rurality                    952
s1 score                    952
s2 score                    952
s3 score                    952
uw gamsat                   952
w gamsat                    952
offer uni                   688
offer uni gpa               688
offer uni gamsat            688
offer uni place type        688
offer uni preference        952
interviewed?                688
interview uni               385
interview uni gpa           385
interview uni gamsat        385
places selected             720
interview uni preference    952
deakin bonus                952
anu bonus                   952
mq bonus (gpa)              952
casper quartile             952
uq rmp tier                 952
gemsas over other?          688
interview opinion           952
notes                       567
year                        952
interview prep hours        509
other rejections            142
marker                      952
dtype: int64

## Interviews

In [19]:
interview_raw = pd.read_csv(f"{RELATIVE_IN}/interview.csv", index_col=0)
print(interview_raw.shape)
print(interview_raw.dtypes)

(1533, 44)
timestamp                object
rurality                 object
s1 score                  int64
s2 score                  int64
s3 score                  int64
uw gamsat               float64
w gamsat                float64
deakin bonus              int64
anu bonus                 int64
mq bonus                  int64
notes                    object
interview?               object
interview uni            object
interview uni gpa       float64
interview uni gamsat    float64
pref 1 uni               object
pref 1 gpa              float64
pref 1 gamsat           float64
pref 2 uni               object
pref 2 gpa              float64
pref 2 gamsat           float64
pref 3 uni               object
pref 3 gpa              float64
pref 3 gamsat           float64
pref 4 uni               object
pref 4 gpa              float64
pref 4 gamsat           float64
pref 5 uni               object
pref 5 gpa              float64
pref 5 gamsat           float64
pref 6 uni               obje

### Fixing Numeric

In [20]:
interview_raw[[col for col in interview_raw.columns if "gamsat" in col or "gpa" in col]].dtypes.to_frame().T

,uw gamsat,w gamsat,interview uni gpa,interview uni gamsat,pref 1 gpa,pref 1 gamsat,pref 2 gpa,pref 2 gamsat,pref 3 gpa,pref 3 gamsat,pref 4 gpa,pref 4 gamsat,pref 5 gpa,pref 5 gamsat,pref 6 gpa,pref 6 gamsat
0,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object


In [21]:
print(interview_raw.shape[0])
print(interview_raw["pref 6 gamsat"].apply(lambda x: isinstance(x, str)).sum())
print(interview_raw["pref 6 gamsat"].apply(lambda x: isinstance(x, int)).sum())
print(interview_raw["pref 6 gamsat"].count())

1533
568
0
568


In [22]:
interview_raw["pref 6 gamsat"][interview_raw["pref 6 gamsat"].notna()].sample(10).to_frame().T

,798,631,671,858,510,923,943,868,721,365
pref 6 gamsat,\n,\n,\n,\n,\n,\n,\n,\n,\n,64.0


In [23]:
interview_raw["pref 6 gamsat"] = interview_raw["pref 6 gamsat"].apply(
    lambda x: x.strip() if isinstance(x, str) else x
)
interview_raw["pref 6 gamsat"] = pd.to_numeric(interview_raw["pref 6 gamsat"])

In [24]:
print(interview_raw["pref 6 gamsat"].count())
print(interview_raw["pref 6 gamsat"].unique())

72
[        nan 64.         63.66666667 62.66666667 66.5        67.75
 63.33333333 66.         68.75       69.         63.5        68.25
 71.25       69.5        70.         65.66666667 71.         63.
 65.75       63.25       67.         65.         61.         69.75
 56.75       61.75       68.         58.25       71.75       66.67
 62.         64.67       62.33       65.33       70.33       64.25
 62.67       68.5        62.75       65.67       62.25       59.
 65.25       61.67       69.25       67.25       52.67       58.        ]


### Switching to Strings

In [25]:
interview_raw.count()

timestamp               1533
rurality                1533
s1 score                1533
s2 score                1533
s3 score                1533
uw gamsat               1532
w gamsat                1533
deakin bonus            1533
anu bonus               1533
mq bonus                1533
notes                    821
interview?              1004
interview uni           1367
interview uni gpa       1367
interview uni gamsat    1367
pref 1 uni               537
pref 1 gpa               537
pref 1 gamsat            537
pref 2 uni               306
pref 2 gpa               306
pref 2 gamsat            306
pref 3 uni               205
pref 3 gpa               205
pref 3 gamsat            205
pref 4 uni               143
pref 4 gpa               143
pref 4 gamsat            143
pref 5 uni               114
pref 5 gpa               114
pref 5 gamsat            114
pref 6 uni                72
pref 6 gpa                72
pref 6 gamsat             72
duplicates?              493
year          

In [26]:
interview_raw.columns

Index(['timestamp', 'rurality', 's1 score', 's2 score', 's3 score',
       'uw gamsat', 'w gamsat', 'deakin bonus', 'anu bonus', 'mq bonus',
       'notes', 'interview?', 'interview uni', 'interview uni gpa',
       'interview uni gamsat', 'pref 1 uni', 'pref 1 gpa', 'pref 1 gamsat',
       'pref 2 uni', 'pref 2 gpa', 'pref 2 gamsat', 'pref 3 uni', 'pref 3 gpa',
       'pref 3 gamsat', 'pref 4 uni', 'pref 4 gpa', 'pref 4 gamsat',
       'pref 5 uni', 'pref 5 gpa', 'pref 5 gamsat', 'pref 6 uni', 'pref 6 gpa',
       'pref 6 gamsat', 'duplicates?', 'year', 'uq tier', 'casper quartile',
       'outlier', 'deakin tier', 'unimelb gam', 'undf bonuses', 'unds bonuses',
       'location', 'uow bonuses'],
      dtype='object')

In [27]:
STRING_COLUMNS = ['deakin bonus', 'anu bonus', 'mq bonus', 'uq tier', 'casper quartile', 'deakin tier', 
                  'unimelb gam', 'undf bonuses', 'unds bonuses', 'uow bonuses']

for string_col in STRING_COLUMNS:
    interview_raw.loc[:, string_col] = interview_raw[string_col].astype(str)
    interview_raw.loc[:, string_col] = offers.replace('nan', 'None')

C:\Users\mtp63\AppData\Local\Temp\ipykernel_8080\1990756901.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0' '2' '0' ... '0' '0' '0']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  interview_raw.loc[:, string_col] = interview_raw[string_col].astype(str)
C:\Users\mtp63\AppData\Local\Temp\ipykernel_8080\1990756901.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0' '4' '0' ... '0' '0' '0']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  interview_raw.loc[:, string_col] = interview_raw[string_col].astype(str)
C:\Users\mtp63\AppData\Local\Temp\ipykernel_8080\1990756901.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0' '0' '0' ... '0' '0' '0']' has dtype incompatible with int

### Changing UQ

In [28]:
UNI_COLUMNS = ["interview uni"] + [f"pref {i+1} uni" for i in range(6)]
UQ_NAME = "The University of Queensland"
RMP_ENDINGS = ["(DD MP)", "(CQ-WB RMP)", "(RMP/DDMP)"]
METRO_ENDING = "(Greater Brisbane)"

# fixing the queensland column
interview = interview_raw.copy()

# change the MD and WB to correct types
for uni_column in UNI_COLUMNS:
    interview.loc[interview[uni_column].isin([f"{UQ_NAME} {x}" for x in RMP_ENDINGS ]), uni_column] = f"{UQ_NAME} (RMP)"
    interview.loc[interview[uni_column] == f"{UQ_NAME} {METRO_ENDING}", uni_column] = f"{UQ_NAME} (Metro)"

# make the notes lowe case
interview.loc[:, "notes"] = interview["notes"].str.lower()

In [29]:
#interview.groupby("interview uni")["year"].value_counts()
interview["interview uni"].value_counts()

interview uni
The University of Melbourne               300
The University of Notre Dame Sydney       198
Deakin University                         126
Griffith University                       110
The University of Western Australia       108
The University of Notre Dame Fremantle    101
The University of Wollongong               93
The University of Queensland (RMP)         83
The University of Queensland               74
Australian National University             62
The University of Queensland (Metro)       60
Macquarie University                       52
Name: count, dtype: int64

In [30]:
interview.to_csv(f"{RELATIVE_OUT}/interview.csv")